In [50]:
import pandas as pd
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import datetime
import numpy as np
from matplotlib import pyplot
from math import sqrt
from numpy import concatenate
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [13]:
#!pip install openpyxl

In [14]:
# Importing Data
df_people = pd.read_csv("full occ csv (1).csv")
df_weather = pd.read_csv("weather-1 (1).csv")
df_electricity = pd.read_excel("consumption.xlsx")

In [15]:
df_people = df_people.dropna(how='all')
df_people =  df_people.drop_duplicates()

In [16]:
df_weather = df_weather.dropna(how='all')
df_weather = df_weather.drop_duplicates()

In [17]:
df_people['Date'] = pd.to_datetime(df_people['Date'], format='%d/%m/%Y')
df_weather['READING DATE'] = pd.to_datetime(df_weather['READING DATE'], format='%Y-%m-%d')

In [18]:
df_people = df_people[df_people['Date'].isin(list(df_weather['READING DATE']))].reset_index().drop(['index'],1)
df_electricity = df_electricity[df_electricity['READING DATE'].isin(list(df_weather['READING DATE']))].reset_index().drop(['index'],1)

In [19]:
df_electricity = df_electricity.drop(["H","OPTIMA Half Hourly DATA","Unnamed: 2","CHANNEL TYPE","UNITS"],axis = 1)

In [20]:
electricity_drop_columns = [list(df_electricity.columns)[i] for i in range(len(list(df_electricity.columns))) if i%2!=0]
df_electricity = df_electricity.drop(electricity_drop_columns,1)
df_electricity['00:00'] = df_electricity[list(df_electricity.columns)[-1]]
df_electricity = df_electricity.drop([list(df_electricity.columns)[-2]],1)

In [21]:
whether_drop_columns = [list(df_weather.columns)[i] for i in range(len(list(df_weather.columns))) if i%2!=0]
df_weather = df_weather.drop(whether_drop_columns,1)
df_weather['00:00'] = df_weather['23:59:00']
df_weather = df_weather.drop(['23:59:00'],1)

In [22]:
for i,j in enumerate(df_people["13:00"]):
    if j == 'UID card access problems':
        df_people["13:00"][i] = np.nan

for k in df_people.columns:
    for i,j in enumerate(df_people[k]):
        if j == 'Sentry Juno installation - Sentry offline, no data 9th -22nd July' or j == 'Sentry Juno live but no data yet' or j == "Sentry Juno put into offline mode" or j == "#" or j == " ":
            df_people[k][i] = np.nan

In [23]:
df_people = df_people.fillna(0)

In [24]:
df_weather = df_weather.fillna(0)

In [25]:
df_people.shape,df_weather.shape,df_electricity.shape

((1520, 25), (1520, 25), (1520, 25))

In [26]:
def get_dataframe(df,date1,name):
    dict1 = {}
    df_col = list(df.columns)[1:]
    for i in range(len(df)):
        for j in range(len(df_col)):
            dict1[str(df.at[i,date1].date()) +' '+ df_col[j]] = df.at[i,df_col[j]]
    return pd.DataFrame(dict1.items(),columns = ['Date',name])

In [27]:
final_df_people = get_dataframe(df_people,'Date','people')
final_df_weather = get_dataframe(df_weather,'READING DATE','weather')
final_df_electricity = get_dataframe(df_electricity,'READING DATE','power_consumption')

In [28]:
final_df_people.shape,final_df_weather.shape,final_df_electricity.shape

((36480, 2), (36480, 2), (36480, 2))

In [29]:
final_df = final_df_people.merge(final_df_weather,on='Date').merge(final_df_electricity,on='Date')

In [30]:
final_df

,Date,people,weather,power_consumption
0,2017-09-01 00:00,0.0,12,0.7
1,2017-09-01 01:00,0.0,12,0.8
2,2017-09-01 02:00,0.0,12,0.8
3,2017-09-01 03:00,0.0,11,0.7
4,2017-09-01 04:00,0.0,11,0.8
...,...,...,...,...
36475,2021-11-13 19:00,230,10,2.1
36476,2021-11-13 20:00,162,9,3.1
36477,2021-11-13 21:00,115,9,2.8
36478,2021-11-13 22:00,67,9,3.2


# Different Approach

In [31]:
temp_df = pd.DataFrame()
for cols in final_df.drop(['Date', 'power_consumption'], axis = 1).columns:
    timeseries_data = final_df[cols]
    scaler = MinMaxScaler()
    print(f'Cols : {cols}')
    scaler.fit(np.array(timeseries_data).reshape(-1, 1))
    scaled_train = scaler.transform(np.array(timeseries_data).reshape(-1, 1))
    n_input = 4
    n_features = 1
    generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size = 64)
    X, y = generator[0]
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape = (X.shape[1], X.shape[2]),return_sequences= True))
    model.add(LSTM(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mse')
    print(f'Training for columns : {cols}')
    model.fit(generator, epochs = 10, verbose = 1)
    test_predictions = []
    first_eval_batch = scaled_train[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    future = 720
    for i in range(future):
        current_pred = model.predict(current_batch)[0]
        test_predictions.append(current_pred)
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

    true_predictions = scaler.inverse_transform(test_predictions)
    temp_df[cols] = true_predictions.flatten()

Cols : people
Training for columns : people
Epoch 1/10
570/570 [==============================] - 103s 180ms/step - loss: 0.0025
Epoch 2/10
570/570 [==============================] - 119s 208ms/step - loss: 7.3022e-04
Epoch 3/10
570/570 [==============================] - 109s 192ms/step - loss: 4.9260e-04
Epoch 4/10
570/570 [==============================] - 105s 185ms/step - loss: 3.6242e-04
Epoch 5/10
570/570 [==============================] - 110s 192ms/step - loss: 3.1919e-04
Epoch 6/10
570/570 [==============================] - 103s 180ms/step - loss: 2.7381e-04
Epoch 7/10
570/570 [==============================] - 86s 151ms/step - loss: 2.6350e-04
Epoch 8/10
570/570 [==============================] - 114s 199ms/step - loss: 2.4697e-04
Epoch 9/10
570/570 [==============================] - 26s 45ms/step - loss: 2.3146e-04
Epoch 10/10
570/570 [==============================] - 12s 20ms/step - loss: 2.2571e-04 1s -
Cols : weather
Training for columns : weather
Epoch 1/10
570/570 [===

In [36]:
temp_df = temp_df.round()
temp_df = abs(temp_df)

,people,weather
0,24.0,8.0
1,33.0,8.0
2,64.0,8.0
3,132.0,8.0
4,243.0,8.0


In [39]:
X = final_df.drop(['Date','power_consumption'],1)
y = final_df['power_consumption']

In [44]:
from sklearn.ensemble import RandomForestRegressor

In [42]:
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.25,random_state=42)
X_train.shape, X_test.shape

((27360, 2), (9120, 2))

In [45]:
rf=RandomForestRegressor()

In [46]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [47]:
pred = rf.predict(X_test)

In [64]:
print("mse : ",mean_squared_error(pred,y_test))

mse :  177.6311343149907


In [55]:
power_consumption_pred = rf.predict(temp_df)

In [60]:
forecast_period_dates = []
date = datetime.datetime(2021,11,13)
for i in range(30): 
    date += datetime.timedelta(days=1)
    for j in range(24):
        forecast_period_dates.append(str(date).split(' ')[0]+' '+str(j)+':00')

In [61]:
temp_df['Date'] = forecast_period_dates

In [58]:
temp_df['power_consumption_pred'] = power_consumption_pred.round()

In [62]:
temp_df

,people,weather,power_consumption_pred,Date
0,24.0,8.0,2.0,2021-11-14 0:00
1,33.0,8.0,2.0,2021-11-14 1:00
2,64.0,8.0,2.0,2021-11-14 2:00
3,132.0,8.0,3.0,2021-11-14 3:00
4,243.0,8.0,3.0,2021-11-14 4:00
...,...,...,...,...
715,112.0,7.0,2.0,2021-12-13 19:00
716,83.0,7.0,2.0,2021-12-13 20:00
717,72.0,7.0,2.0,2021-12-13 21:00
718,81.0,7.0,3.0,2021-12-13 22:00


In [ ]:
temp_df = temp_df.round()
temp_df = abs(temp_df)
final_df_forecasted = temp_df.astype(int)

In [ ]:
final_df_forecasted['power_consumption'] = np.zeros(720)

In [ ]:
final_df

In [ ]:
power_consumption_lst = final_df['power_consumption']
final_df.drop(['power_consumption'], axis = 1, inplace = True)

final_df['power_consumption'] = power_consumption_lst

In [ ]:
final_df_forecasted = pd.concat([final_df, final_df_forecasted])

In [ ]:
final_df_forecasted

In [ ]:
df_temp = final_df_forecasted.copy()

In [ ]:
final_df_forecasted.drop(['Date'], axis = 1, inplace = True)

In [ ]:
final = final_df_forecasted.copy()

In [ ]:
final.shape

### Forecasting power consumption

In [ ]:
scaler_std = StandardScaler()
scaler_std = scaler_std.fit(final_df_forecasted)
final_df_trainscaled = scaler_std.transform(final_df_forecasted)

In [ ]:
Train_X = []
Train_y = []

n_Future = 720
n_Past = 1440

In [ ]:
#iterate between the past observations limit and the length of the df and subtract the future(days to predict into the future) and add 1 
for i in range(n_Past, len(final_df_trainscaled) - n_Future + 1):
    #append each value to Train_X
    Train_X.append(final_df_trainscaled[i - n_Past:i, 0:final_df.shape[1]])
    #append CycleTime to the Train_y list [-1] is the col index of Cycletime
    Train_y.append(final_df_trainscaled[i + n_Future - 1:i + n_Future, -1])

In [ ]:
Train_X, Train_y = np.array(Train_X), np.array(Train_y)

In [ ]:
print(f'Train X shape : {Train_X.shape}')
print(f'Train y shape : {Train_y.shape}')

In [ ]:
#create the DL model
model = Sequential()
#using relu as the activation
model.add(LSTM(64, activation='relu', input_shape = (Train_X.shape[1], Train_X.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(Train_y.shape[1]))

#using adam as the activation
model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

In [ ]:
history = model.fit(Train_X, Train_y, 
                    epochs = 1, batch_size = 32,
                    verbose = 1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
N_past = 721
N_future = 720

In [ ]:
forecast_period_dates = []
date = datetime.datetime(2021,11,13)
for i in range(30): 
    date += datetime.timedelta(days=1)
    for j in range(24):
        forecast_period_dates.append(str(date).split(' ')[0]+' '+str(j)+':00')

In [ ]:
#predict using the observations from the previous data
prediction = model.predict(Train_X[-N_future:])

In [ ]:
prediction

In [ ]:
#repeat the predictions for the fututre days(given as input)
predictions = np.repeat(prediction, final_df_forecasted.shape[1], axis=-1)
#inverse_transform
y_pred_future = scaler_std.inverse_transform(predictions)[:,0]

In [ ]:
# Append the predictions to the list
 
df_fr_final = pd.DataFrame({'Date':np.array(forecast_period_dates), 'power_consumption':y_pred_future})
df_fr_final['Date'] = pd.to_datetime(df_fr_final['Date'])

In [ ]:
df_fr_final['Date'] = pd.to_datetime(df_fr_final['Date'])

In [ ]:
import seaborn as sns

plt.figure(figsize = (15,4))
ax = sns.lineplot(df_fr_final['Date'], df_fr_final['power_consumption'])

In [ ]:
df_fr_final['people'] = temp_df['people']
df_fr_final['weather'] = temp_df['weather']

In [ ]:
df_fr_final

# Different Approach

In [ ]:
def get_predicted_x_df(df):
    #creating empty dataframe
    temp_df = pd.DataFrame()

    #running for loop on two columns people and whether to run LSTM on them to predict future values
    for cols in df.drop(['Date'], axis = 1).columns:
        timeseries_data = df[cols]
        scaler = MinMaxScaler()
        print(f'Cols : {cols}')
        scaler.fit(np.array(timeseries_data).reshape(-1, 1))
        scaled_train = scaler.transform(np.array(timeseries_data).reshape(-1, 1))
        n_input = 10
        n_features = 1
        generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size = 64)
        X, y = generator[0]
        model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape = (X.shape[1], X.shape[2]),return_sequences= True))
        model.add(LSTM(32, activation='relu', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(32, activation='relu'))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer = 'adam', loss = 'mse')
        print(f'Training for columns : {cols}')
        model.fit(generator, epochs = 10, verbose = 1)
        test_predictions = []
        first_eval_batch = scaled_train[-n_input:]
        current_batch = first_eval_batch.reshape((1, n_input, n_features))
        future = 30 # this will predict no. of future values.
        for i in range(future):
            current_pred = model.predict(current_batch)[0]
            test_predictions.append(current_pred)
            current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

        true_predictions = scaler.inverse_transform(test_predictions)
        temp_df[cols] = true_predictions.flatten()
    return temp_df

In [ ]:
predicted_df_x_people = get_predicted_x_df(df_people)
predicted_df_x_weather = get_predicted_x_df(df_weather)

In [ ]:
mse1 = {}
train_df = pd.DataFrame()
prediction_df = pd.DataFrame()
predicted_df_y_power_consumption = pd.DataFrame()
#getting future date hour values
forecast_period_dates = []
date = datetime.datetime(2021,11,13)
for i in range(30): 
    date += datetime.timedelta(days=1)
    forecast_period_dates.append(date)
predicted_df_y_power_consumption['Dates'] = forecast_period_dates
predicted_df_x_people['Dates'] = forecast_period_dates
predicted_df_x_weather['Dates'] = forecast_period_dates

In [ ]:
for i in list(predicted_df_x_people.columns):
    print(i)
    train_df['people_'+i]  =  df_people[i]
    train_df['weather_'+i]  =  df_weather[i]
    print('people_'+i,'weather_'+i)
    X = train_df[['people_'+i,'weather_'+i]]
    y = df_electricity[i]
    #Scaling the Data
    scaler=StandardScaler()
    col=X.columns
    X=scaler.fit_transform(X)
    X=pd.DataFrame(X,columns=col)
    X_train, X_test, y_train, y_test = tts(X,y,test_size=0.25,random_state=42)
    X_train.shape, X_test.shape
    rf=RandomForestRegressor(n_estimators = 200, min_samples_split = 5,min_samples_leaf = 4,max_features = 'auto',max_depth = 10,bootstrap = True   )
    rf.fit(X_train,y_train)
    pred = rf.predict(X_test)
    mse1[i] = mean_squared_error(pred,y_test)  
    prediction_df['people_'+i]  =  predicted_df_x_people[i]
    prediction_df['weather_'+i]  =  predicted_df_x_weather[i]
    power_consumption_pred = rf.predict(prediction_df[['people_'+i,'weather_'+i]])
    predicted_df_y_power_consumption[i] = power_consumption_pred
    print("done")

In [ ]:
predicted_df_y_power_consumption

# Different Approach

In [ ]:
final_df1 = final_df[['people']]

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
values = final_df1.values
values = values.astype('float32')
reframed = series_to_supervised(values, 30, 1)
scaler = MinMaxScaler()#scaler.inverse_transform
scaled = scaler.fit_transform(reframed)

In [ ]:
# split into train and test sets
values = scaled
n_train_hours = 27360
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=32, validation_data=(test_X, test_y), verbose=1, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
test_X.shape

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, :]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, :]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
# calculate RMSE
mse = mean_squared_error(inv_y, inv_yhat)
print('Test MSE: %.3f' % mse)